# 📌 Connexion au Notebook avec Singularity

## 🔹 Image Singularity  
Lancer l'image Singularity suivante :  
`6_Containers/shortcakelight.sif`  

[ShortCake](https://github.com/rnakato/ShortCake/tree/master) est conçu spécifiquement pour les analyses de données single-cell.  

---

## 🚀 Lancement de l'image et connexion au Jupyter Notebook  

### Lancer l'image Singularity  
```bash
srun --pty bash
singularity exec 6_Containers/shortcakelight.sif jupyternotebook.sh
```
##3 Puis faire le lien ssh sur l'ordi avec cette commande que l'on écrit dans le terminal local:
```bash
ssh -A -t -t vgoupille@genossh.genouest.org -L 8888:localhost:8888 ssh cl1n030 -L 8888:localhost:8888
modifier le port (numero et noeud si besoin)
```

## si probleme de connection 

```bash
ls -a
rm -r .vscode-server
#puis se remet sur vscode 
# et refait la liaison avec ssh remote (et reinstaller les extensions...)
```

Ce texte décrit les stratégies et les méthodes utilisées pour traiter et aligner les données issues de l’expérience scRNA-seq réalisée avec la méthode microSPLiT.

1. **Stratégie de séquençage (Library strategy)** :
	- La méthode utilisée est microSPLiT, qui est une adaptation de la technique SPLiT-seq, permettant de réaliser du scRNA-seq sans nécessiter d’isolation physique des cellules.

2. **Prétraitement et alignement des données** :
	- L’alignement des séquences a été effectué avec une version modifiée du pipeline SPLiT-seq disponible sur GitHub (https://github.com/Alex-Rosenberg/split-seq-pipeline).
	- Modifications apportées :
	  - Utilisation de STAR (un logiciel d’alignement des lectures RNA-seq) avec une détection des isoformes d’épissage.
	  - Sélection des meilleurs alignements multimapping selon un critère de score élevé.
	  - Conservation d’un compte fonctionnel basé sur le nombre d’alignements de bonne qualité, ce qui est pertinent pour les génomes bactériens où plusieurs CDSs (séquences codantes) peuvent se chevaucher.

3. **Références génomiques utilisées** :
	- Les génomes de référence utilisés pour l’alignement sont :
	  - ASM904v1.45
	  - ASM80076v1.37
	- Ces versions proviennent de la base de données EnsemblBacteria.

4. **Fichiers supplémentaires et leur contenu** :
	- Les fichiers supplémentaires fournis incluent :
	  - Matrice cellule-gène (contenant uniquement les cellules ayant un minimum de 200 UMI/cellule).
	  - Noms des gènes.
	  - Annotations des cellules :
		 - Pour les données du stress thermique (heat-shock) : les cellules sont annotées par code-barres et puits.
		 - Pour la courbe de croissance de Bacillus subtilis : les cellules sont annotées selon leur densité optique (OD). Seuls les ARNm (mRNA) sont inclus dans la matrice cellule-gène.

5. **Organisation des cellules dans l’expérience de stress thermique** :
	- Les cellules sont disposées dans 48 puits, avec une distinction entre conditions :
	  - Puits 1-24 : cellules soumises au stress thermique.
	  - Puits 25-48 : cellules témoins (non soumises au stress thermique).

replica 2 : M14 => B. subtilis PY79 (OD0.5-OD3.2)

replica 1 : M15 => B. subtilis PY79 (OD0.5-OD6.0)

other plate : M11 => B. subtilis PY79 + E. coli MW1255

# Telechargement des données preocessées  de l'article de Kuchina et al: 
Microbial single-cell RNA sequencing by split-pool barcoding
https://doi.org/10.1126/science.aba5257

protocole 2024: 
https://doi.org/10.1038/s41596-024-01007-w

les données de séquençage brute sont disponible sur SRA : 
The raw sequencing files are available at the Sequence Read Archive:
    - GSM4594094 : https://www.ncbi.nlm.nih.gov/sra/SRX8485151%5baccn%5d
    - GSM4594095 : https://www.ncbi.nlm.nih.gov/sra/SRX8485152%5baccn%5d
    - GSM4594096 : https://www.ncbi.nlm.nih.gov/sra/SRX8485153%5baccn%5d

Processed data were submitted to Gene Expression Omnibus, with accession number GSE151940.

#Les données processées sont disponibles sur le site de GEO:
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE151940


SUPPLEMENTARY MATERIALS 
https://www.science.org/doi/10.1126/science.aba5257


NOUS N'ALLONS PAS UTILISER C'EST DONNEES QUI NE SONT PAS OBTENUE AVEC STARsolo 



Un autre article a analysé les données de scRNA-seq de la même expérience.
"Single-cell heterogeneity in ribosome content and the  consequences for the growth laws" par Brettner et al. (2024).

https://pmc.ncbi.nlm.nih.gov/articles/PMC11185559/

Récuperation des données et divers script sur la page :
    https://osf.io/kjfbz/
    
    - donnée Raw Data : obtenue avec STARsolo
    GEO060 => M14
    GEO061 => M15
    

Import libraries, usefull functions, and data 

In [1]:
# Chargement des librairies

library(reticulate) # Permet d'utiliser des fonctions python dans R
library(Seurat) # Permet de manipuler des données de single-cell RNA-seq
library(ggplot2) # Permet de faire des graphiques
library(dplyr) # Permet de manipuler des données
library(Matrix) # Permet de manipuler des matrices
library(cowplot) # Permet de faire des graphiques
library(forcats) # Permet de manipuler des facteurs
library(RColorBrewer) # Permet de manipuler les couleurs

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
# Chargement des fonctions R 

round_one_bc_collapse <- function(data, threshold){
  
  r6pt <- read.csv('7_Article/script/utile_bact/r6ptorderedbcs.csv')
  r6pt <- r6pt$x
  
  temp <- data[,r6pt]
 
  temp <- temp[,which(c(1:length(r6pt))%%2==1)] + temp[,which(c(1:length(r6pt))%%2==0)]
 
 colnames(temp) <- r6pt[which(c(1:length(r6pt))%%2==1)]
 rownames(temp) <- rownames(data)
 
 temp <- temp[,which(colSums(temp) > 0)]
 
 y = as.numeric(log(colSums(temp)[order(colSums(temp), decreasing = TRUE)]))
 x = log(c(1:length(y)))
 plot(x,y)

 slope = as.numeric((y[length(y)]-y[1])/(x[length(x)]-x[1]))
 yline = slope*x + y[1]
 lines(x,yline, col = 2)

 d = sqrt((y-yline)^2)
 plot(x,d)
 d2 = d[which(d >= d[which.max(d)]*threshold)]
 x2 = x[which(d >= d[which.max(d)]*threshold)]
 points(x2,d2, col = 3)
 
 temp <- temp[,which(colSums(temp) >= exp(y[which(x == x2[which.max(x2)])]))]
 #dim(temp)
 
 return(temp)
}



# Cette fonction traite des données de séquençage à haut débit, en particulier pour consolider et filtrer les codes-barres selon leur abondance. Voici son fonctionnement étape par étape:

# 1. **Chargement des codes-barres organisés**: La fonction commence par charger un fichier CSV prédéfini qui contient une liste ordonnée de codes-barres (au format BC3_BC2_BC1).

# 2. **Réorganisation des données**: Elle sélectionne les colonnes de l'ensemble de données d'entrée qui correspondent aux codes-barres du fichier CSV.

# 3. **Consolidation des paires**: La fonction additionne les données des colonnes paires et impaires. Ceci combine les lectures provenant de paires de codes-barres qui correspondent au même puits physique (comme visible dans votre premier script où chaque puits a deux codes-barres alternatifs).

# 4. **Nettoyage des données**:
#    - Elle renomme les colonnes pour n'utiliser que les codes-barres impairs
#    - Elle préserve les noms des lignes du jeu de données original
#    - Elle supprime les colonnes dont la somme est zéro (codes-barres sans lectures)

# 5. **Analyse statistique pour déterminer un seuil de filtrage**:
#    - Elle calcule le logarithme de la somme des lectures pour chaque code-barre
#    - Elle trace un graphique log-log du rang des codes-barres vs leur abondance
#    - Elle calcule une ligne de tendance linéaire sur ce graphique
#    - Elle mesure la distance entre chaque point et cette ligne de tendance
#    - Elle identifie les points dont la distance dépasse un certain pourcentage (`threshold`) de la distance maximale

# 6. **Filtrage final**: La fonction ne conserve que les codes-barres dont le nombre de lectures dépasse le seuil déterminé statistiquement.

# 7. **Retour des résultats**: Elle renvoie le jeu de données filtré et consolidé.

# Cette approche utilise une méthode statistique appelée "knee point detection" (détection du point d'inflexion) pour déterminer automatiquement le seuil optimal de filtrage des codes-barres. Cela permet d'éliminer les codes-barres de faible qualité ou peu abondants tout en conservant ceux qui sont statistiquement significatifs, ce qui est crucial dans les analyses de séquençage à cellule unique.

# Le code commenté (plus long) semble être une version antérieure qui effectuait manuellement l'attribution des codes-barres aux puits, similaire à la fonction `assign_cell_wells` que vous avez partagée précédemment. La version actuelle simplifie ce processus en utilisant un fichier CSV préexistant.


















# Fonction pour créer un objet Seurat à partir de fichiers de séquençage
create_seurat_object_from_seq_files <- function(data_dir, sublibrary, ribosome_removal, threshold) {
  
  # Vérifier si le répertoire de données existe
  if (!dir.exists(data_dir)) {
    stop("Le répertoire spécifié n'existe pas : ", data_dir)
  }
  
  # Construire les chemins des fichiers
  matrix_file <- file.path(data_dir, "UniqueAndMult-Uniform.mtx")
  features_file <- file.path(data_dir, "features.tsv")
  barcodes_file <- file.path(data_dir, "barcodes.tsv")

  # Vérifier si les fichiers existent
  if (!file.exists(matrix_file) || !file.exists(features_file) || !file.exists(barcodes_file)) {
    stop("Un ou plusieurs fichiers de données sont introuvables dans : ", data_dir)
  }

  # Lecture des fichiers de comptage
  data <- readMM(matrix_file)
  genes <- read.table(features_file, sep = "\t", header = FALSE, stringsAsFactors = FALSE)$V1
  genes <- convert_gene_names_bacteria(genes)
  barcodes <- read.table(barcodes_file, sep = "\t", header = FALSE, stringsAsFactors = FALSE)$V1
  
  # Structuration des données
  rownames(data) <- genes
  colnames(data) <- barcodes

  # Filtrage des cellules
  filteredData <- round_one_bc_collapse(data, threshold)

  # Gestion des reads ribosomiques si nécessaire
  if (ribosome_removal == 'rRNA') {
    # Déterminer le chemin du fichier de ribosomes
    ribosome_file <- file.path("7_Article/script/utile_bact/bacteria.ribosomes.csv")
    
    if (!file.exists(ribosome_file)) {
      stop("Le fichier de ribosomes est introuvable : ", ribosome_file)
    }
    
    ribosomes <- read.csv(ribosome_file, stringsAsFactors = FALSE)
    colnames(ribosomes) <- c('gene', 'type')
    rRNA <- ribosomes[ribosomes$type == 'rRNA', ]
    mRNA <- filteredData[!(rownames(filteredData) %in% rRNA$gene), ]
    
    # Création de l'objet Seurat
    SO <- CreateSeuratObject(counts = mRNA)
  } else {
    # Création de l'objet Seurat sans suppression des reads ribosomiques
    SO <- CreateSeuratObject(counts = filteredData, min.cells = 15, min.features = 5)
  }
  
  # Ajout de métadonnées
  wells <- assign_cell_wells(colnames(SO@assays$RNA))
  celldata <- as.data.frame(wells$well)
  row.names(celldata) <- wells$barcode
  
  SO$well <- celldata$V1
  SO$sublibrary <- sublibrary
  
  return(SO)
}

In [6]:
create_seurat_object_from_seq_files <- function(data_dir, sublibrary, ribosome_removal, threshold = NULL, ribosome_file = NULL) {
  
  # Vérifier si le répertoire de données existe
  if (!dir.exists(data_dir)) {
    stop("Le répertoire spécifié n'existe pas : ", data_dir)
  }
  
  # Construire les chemins des fichiers
  matrix_file <- file.path(data_dir, "UniqueAndMult-Uniform.mtx")
  features_file <- file.path(data_dir, "features.tsv")
  barcodes_file <- file.path(data_dir, "barcodes.tsv")

  # Vérifier si les fichiers existent
  if (!file.exists(matrix_file) || !file.exists(features_file) || !file.exists(barcodes_file)) {
    stop("Un ou plusieurs fichiers de données sont introuvables dans : ", data_dir)
  }

  # Lecture des fichiers de comptage
  data <- readMM(matrix_file)
  genes <- read.table(features_file, sep = "\t", header = FALSE, stringsAsFactors = FALSE)$V1
  genes <- convert_gene_names_bacteria(genes)
  barcodes <- read.table(barcodes_file, sep = "\t", header = FALSE, stringsAsFactors = FALSE)$V1
  
  # Structuration des données
  rownames(data) <- genes
  colnames(data) <- barcodes

  # Filtrage des cellules avec ou sans threshold
  if (!is.null(threshold)) {
    filteredData <- round_one_bc_collapse(data, threshold)
  } else {
    filteredData <- round_one_bc_collapse(data)
  }

  # Gestion des reads ribosomiques si nécessaire
  if (ribosome_removal == 'rRNA') {
    # Vérifier que le fichier de ribosomes est fourni
    if (is.null(ribosome_file) || !file.exists(ribosome_file)) {
      stop("Le fichier de ribosomes est introuvable ou non spécifié : ", ribosome_file)
    }
    
    ribosomes <- read.csv(ribosome_file, stringsAsFactors = FALSE)
    colnames(ribosomes) <- c('gene', 'type')
    rRNA <- ribosomes[ribosomes$type == 'rRNA', ]
    mRNA <- filteredData[!(rownames(filteredData) %in% rRNA$gene), ]
    
    # Création de l'objet Seurat
    SO <- CreateSeuratObject(counts = mRNA)
  } else {
    # Création de l'objet Seurat sans suppression des reads ribosomiques
    SO <- CreateSeuratObject(counts = filteredData, min.cells = 15, min.features = 5)
  }
  
  # Ajout de métadonnées
  wells <- assign_cell_wells(colnames(SO@assays$RNA))
  celldata <- as.data.frame(wells$well)
  row.names(celldata) <- wells$barcode
  
  SO$well <- celldata$V1
  SO$sublibrary <- sublibrary
  
  return(SO)
}

In [ ]:
# Chargement des données STARsolo pour les deux réplicats
# avec chacun  :
# barcodes.tsv => les codes-barres des cellules
# features.tsv => les gènes

# UniqueAndMult-Uniform.mtx => les comptages de gènes par cellule (ici en format Matrix Market) et contient les allignements uniques et multiples des reads (un read peut être aligné à plusieurs endroits dans le génome)
# matrix.mtx => les comptages de gènes par cellule (ici en format Matrix Market) et contient les allignements uniques des reads (un read est aligné à un seul endroit dans le génome)

# Replica n°1 : M15 (GEO61)
replica1 <- '7_Article/data/data_osf/GEO061'
# Replica n°2 : M14 (GEO60)
replica2 <- '7_Article/data/data_osf/GEO060'


In [ ]:
create_seurat_object_from_seq_files(replica1, 'M15', ribosome_removal = 'rRNA', threshold = 0.1)